# Hello World

In [ ]:
# !python -m pip install -U crewai crewai-tools serpapi

In [ ]:
import os
# Set up API keys


In [ ]:
import os
from crewai import Agent, Task, Crew
# Importing crewAI tools

from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerpApiGoogleSearchTool,
    WebsiteSearchTool
)


In [ ]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./')
file_tool = FileReadTool()
search_tool = SerpApiGoogleSearchTool()
web_rag_tool = WebsiteSearchTool()


In [ ]:
docs_tool

In [ ]:
search_tool.run(search_query='sinan ozdemir')

In [ ]:
# Create agents
researcher = Agent(
    role='Market Research Analyst',
    goal='Provide up-to-date market analysis of the AI industry',
    backstory='An expert analyst with a keen eye for market trends.',
    tools=[search_tool, web_rag_tool],
    verbose=True
)

writer = Agent(
    role='Content Writer',
    goal='Craft engaging blog posts about the AI industry',
    backstory='A skilled writer with a passion for technology.',
    tools=[docs_tool, file_tool],
    verbose=True
)

# Define tasks
research = Task(
    description='Research the latest trends in the AI industry in 2025 and provide a summary.',
    expected_output='A summary of the top 3 trending developments in the AI industry with a unique perspective on their significance.',
    agent=researcher
)

write = Task(
    description='Write an engaging blog post about the AI industry in 2025, based on the research analyst\'s summary. Draw inspiration from the latest blog posts in the directory.',
    expected_output='A 4-paragraph blog post formatted in markdown with engaging, informative, and accessible content, avoiding complex jargon.',
    agent=writer,
    output_file='blog-posts/new_post.md'  # The final blog post will be saved here
)


In [ ]:
# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=True,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

In [ ]:
# Assemble a crew with planning enabled
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=True,
    planning=False,  # Enable planning feature
)

# Execute tasks
crew.kickoff()

In [ ]:
from crewai import Process
data_analyst = Agent(
    role='Educator',
    goal=f'Based on the context provided, answer the question.',
    backstory='You are a data expert',
    verbose=True,
    allow_delegation=False, #Disable delegation to another agent
    tools=[search_tool]
)

test_task = Task(
    description="Understand the topic of Natural Language Processing and summarize it for me",
    agent=data_analyst,
    expected_output='I want the response to be as short as possible.'
)

crew = Crew(
    tasks=[test_task],
    process=Process.sequential,
)

output = crew.kickoff()

In [ ]:
print(output.raw)

In [ ]:
from langchain_openai import ChatOpenAI
import litellm
from crewai import Agent, Task, Crew
from typing import Dict, Any

class LLMCallTracker:
    def __init__(self):
        self.call_count = 0
        self.calls = []
    
    def track_call(self, **kwargs):
        self.call_count += 1
        call_info = {
            'call_number': self.call_count,
            'timestamp': datetime.now(),
            'model': kwargs.get('model'),
            'messages': kwargs.get('messages'),
            'response': kwargs.get('response')
        }
        self.calls.append(call_info)
        
        print(f"\n=== LLM Call #{self.call_count} ===")
        print(f"Model: {call_info['model']}")
        print(f"Messages: {len(call_info['messages'])} messages")
        print(f"Response length: {len(str(call_info['response']))} chars")
        print("=" * 50)

# Enable LiteLLM debug with custom tracking
tracker = LLMCallTracker()
litellm._turn_on_debug()

nlp_task = Task(
    description="Understand the topic of Natural Language Processing and sumamrize it for me",
    # agent=data_analyst,  # commenting out the agent now, letting the manager decide
    expected_output='Give a correct response'
)

calculator_agent = Agent(
    role='Calculator',
    goal=f'You calculate things',
    backstory='You love math',
    verbose=True,
    allow_delegation=False,
    tools=[]
)

math_task = Task(
    description="Tell me what 123 * 34 is",
    expected_output='Calculate this'
)


crew = Crew(
    tasks=[math_task, nlp_task],
    agents=[calculator_agent, data_analyst],
    process=Process.hierarchical,
    manager_llm=ChatOpenAI(model_name="gpt-4.1", temperature=0.7),  # required if you are using hierarhical
    verbose=True
)

output = crew.kickoff()

In [ ]:
for output in output.tasks_output:
    print(output)
    print('----')

In [ ]:
output.raw

In [ ]:
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science',
  backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool]
)

writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
  verbose=True,
  allow_delegation=True
)

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2025.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
from crewai import Process
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  # When delegation enabled in writer agent, Crew auto finds required agent for task2 & task1 is not requied to be mentioned
  # Also auto corrects parameters required for tool delegation
  tasks= [task2], #[task1, task2],
  process=Process.sequential,
)

In [ ]:
# writes tries to use delegate to worker wrong. Fixes itself
result = crew.kickoff()

print("-----------------------------")
print(result)


In [ ]:
writer.allow_delegation = False  # turn off delegation, resulting in all hard-coded delegation

In [ ]:
# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
)
result = crew.kickoff()
print("-----------------------------")
print(result)

In [ ]:
print(task1.output.raw)

In [ ]:
print(task2.output.raw)

# Building a Search Agent with a Custom Tool

In [ ]:
os.environ["SERP_API_KEY"] = 'your_serp_key'

In [ ]:
!uv pip show serpapi

In [ ]:
!uv pip install --upgrade serpapi

In [ ]:
import json
import os
import requests
from crewai import Agent, Task
# from serpapi import GoogleSearch
from crewai.tools import BaseTool

class SearchTool(BaseTool):
    name: str = "Google Search"
    description: str = """Useful to search the internet about a given topic and return relevant results."""

    def _run(self, query: str, top_k: int = 3) -> str:
        params: dict = {
            "engine": "google",
            "google_domain": "google.com",
            "gl": "us",
            "hl": "en",
            "q": query,
            "api_key": os.environ["SERP_API_KEY"],
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        # check if there is an organic results key, don't include sponsor results
        if 'organic_results' not in response:
            return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
        else:
            results = response['organic_results']
            string = []
            for result in results[:top_k]:
                try:
                    string.append('\n'.join([
                        f"Title: {result['title']}", f"Link: {result['link']}",
                        f"Snippet: {result['snippet']}", "\n-----------------"
                    ]))
                except KeyError:
                    next

            return '\n'.join(string)

In [ ]:
SearchTool().description # where I specifically describe how to use the tool

In [ ]:
print(SearchTool().run(query="Who is Sinan Ozdemir?"))

In [ ]:
researcher_agent = Agent(
    role='Researcher',
    goal='Look up information on the internet and scrape the websites for relevant information',
    backstory="""You have a knack for dissecting complex data and presenting actionable insights.""",
    tools=[
        SearchTool()
    ],
    verbose=True
)
email_agent = Agent(
    role='Emailer',
    goal='Write short, warm emails to people',
    backstory="""You are an assistant.""",
    tools=[],
    verbose=True
)

sample_lookup_task = Task(
  description="""Lookup Sinan Ozdemir""",
  expected_output="A lookup report of a query",
  agent=researcher_agent
)

email_task = Task(
  description="""Write an email to Sinan Ozdemir asking him to get coffee. Personalize it""",
  expected_output="A short warm email",
  agent=email_agent
)

crew = Crew(
    tasks=[sample_lookup_task, email_task],
    process=Process.sequential,
    verbose=True
)

result = crew.kickoff()

In [ ]:
print(result)

## Trip Advisor

In [ ]:
from crewai import Agent

class TripAgents():

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide 2-3 insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            search_tool, #SearchTool(),
            web_rag_tool
        ],
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with
        decades of experience""",
        tools=[],
        verbose=True)

In [ ]:
from crewai import Task
from textwrap import dedent
from datetime import date


class TripTasks():

  def gather_task(self, agent, origin, city, interests, range):
    return Task(description= dedent(f"""
        I want you to create a comprehensive city guide.
        This guide should provide a thorough overview of what
        the city has to offer, including hidden gems, cultural
        hotspots, and must-visit landmarks.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
        """),

            expected_output=dedent(f"""
        The final answer must be a comprehensive city guide,
        rich in cultural insights and practical tips,
        tailored to enhance the travel experience.

        """),
                agent=agent)

  def plan_task(self, agent, origin, city, interests, range):
    return Task(description=dedent(f"""
        Expand this guide into a a full 1-day travel
        itinerary with detailed per-day plans, including
        weather forecasts, places to eat, packing suggestions,
        and a budget breakdown.

        You MUST suggest actual places to visit, actual hotels
        to stay and actual restaurants to go to.

        Trip Date: {range}
        Traveling from: {origin}
        Traveling to: {city}
        Traveler Interests: {interests}
      """),
            expected_output=dedent(f"""
        Your final answer MUST be a complete expanded travel plan,
        formatted as markdown, encompassing a daily schedule,
        anticipated weather conditions, recommended clothing and
        items to pack, and a detailed budget, ensuring THE BEST
        TRIP EVER, Be specific and give it a reason why you picked
        # up each place, what make them special!
        """),
                agent=agent)

In [ ]:
class TripCrew:

  def __init__(self, origin, city, date_range, interests):
    self.city = city
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    gather_task = tasks.gather_task(
      agents.local_expert(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      agents.travel_concierge(),
      self.origin,
      self.city,
      self.interests,
      self.date_range
    )

    # Initiate Crew
    crew = Crew(
      tasks=[gather_task, plan_task],
      process=Process.sequential,
      verbose=True
    )

    result = crew.kickoff()
    return result

In [ ]:
trip_crew = TripCrew('SF', "Paris", 'December 2025', 'art')
result = trip_crew.run()

In [ ]:
print(result)

In [ ]:
print("## Welcome to Trip Planner Crew")
print('-——————————')
location = input(
dedent("""
  From where will you be traveling from?
"""))
city = input(
dedent("""
  Where do you want to go?
"""))
date_range = input(
dedent("""
  What is the date range you are interested in traveling?
"""))
interests = input(
dedent("""
  What are some of your high level interests and hobbies?
"""))

trip_crew = TripCrew(location, city, date_range, interests)
result = trip_crew.run()

In [ ]:
print("\n\n########################")
print("## Here is your Trip Plan")
print("########################\n")
print(result)